### How to use few shot examples in chat models
https://python.langchain.com/docs/how_to/few_shot_examples_chat/

질문과 답변의 쌍을 여러개 예시로 만들어, 질의할 때 같이 포함하여 답변의 정확도를 크게 끌어올린다.

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import langchain_core, langchain_ollama

#langchain_core : 0.3.9
#langchain_ollama : 0.2.0
print("langchain_core : " + langchain_core.__version__)
print("langchain_ollama : " + langchain_ollama.__version__)

langchain_core : 0.3.9
langchain_ollama : 0.2.0


In [29]:
llm = ChatOllama(
    model="gemma2:9b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
    # other params...
)

In [28]:
first_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        ("human", "{input}")
    ]
)

chain1 = first_prompt | llm

In [24]:
ai_msg1 = chain1.invoke({"input": "What is 8 🍎 9?"})

In [25]:
ai_msg1.content

'Ah, a delightful riddle!  \n\nIn the realm of numbers, "🍎" usually means addition. So, 8 🍎 9 is simply:\n\n8 + 9 = 17 \n\n\nLet me know if you have any other magical mathematical puzzles for me! ✨🧮✨'

### How to use few shot examples

In [6]:
examples = [
    {"input": "2 🍎 4", "output": "8"},
    {"input": "3 🍎 5", "output": "15"},
    {"input": "7 🍎 3", "output": "21"}
]

프롬프트 템플릿을 생성한다.

In [17]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)
example_prompt

ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})])

In [18]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [19]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': '2 🍎 4', 'output': '8'}, {'input': '3 🍎 5', 'output': '15'}, {'input': '7 🍎 3', 'output': '21'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [21]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [22]:
chain2 = final_prompt | llm

In [23]:
ai_msg2 = chain2.invoke({"input": "What is 8 🍎 9?"})

In [26]:
ai_msg2

AIMessage(content='72 \n\nRemember, "🍎" means multiplication!  😊  \n', additional_kwargs={}, response_metadata={'model': 'gemma2:9b', 'created_at': '2024-10-10T23:58:43.416877Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6294954000, 'load_duration': 3631200750, 'prompt_eval_count': 77, 'prompt_eval_duration': 923750000, 'eval_count': 17, 'eval_duration': 1733492000}, id='run-60626c5b-5f56-4159-a2b3-5c81c7e2b28f-0', usage_metadata={'input_tokens': 77, 'output_tokens': 17, 'total_tokens': 94})

### Dynamic few-shot prompting

여러가지 질의-답변 쌍중에 관련있는 예제만 k 개 추출하여 모델에 넣어 답변의 정확도를 높이는 방법.
여기서는 Chroma 벡터DB로 검색.

In [111]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [112]:
llm = ChatOllama(
    model="gemma2:9b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
    # other params...
)

In [114]:
examples = [
    {"input": "2 🍎 4", "output": "8"},
    {"input": "3 🍎 5", "output": "15"},
    {"input": "7 🍎 3", "output": "21"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

In [115]:
to_vectorize = [" ".join(example.values()) for example in examples]

In [116]:
to_vectorize

['2 🍎 4 8',
 '3 🍎 5 15',
 '7 🍎 3 21',
 'What did the cow say to the moon? nothing at all',
 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']

In [117]:
embeddings = OllamaEmbeddings(model="gemma2:9b", 
                             base_url="http://host.docker.internal:11434")

In [118]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [119]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore, k=2
)

In [120]:
example_selector.select_examples({"input": "horse"})

[{'input': 'Write me a poem about the moon',
  'output': 'One for the moon, and one for me, who are we to talk about the moon?'},
 {'input': 'What did the cow say to the moon?', 'output': 'nothing at all'}]

3 🦜 3 이라는 인풋값을 넣으면 그에 맞는 예시를 추출하게 된다. 사실 수많은 예시중에 모델에 넣을 예시를 선택하는 것도 LLM 을 사용하게 될줄은 몰랐다.

In [121]:
example_selector.select_examples({"input": "3 🦜 3"})

[{'input': '3 🍎 5', 'output': '15'}, {'input': '7 🍎 3', 'output': '21'}]

### Create prompt template

이제 실제 질의할 프롬프트를 만들어 실행해보자.

In [122]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [143]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    )
)

In [141]:
few_shot_prompt

FewShotChatMessagePromptTemplate(example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0xffff545e0ad0>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None), input_variables=['input2'], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [142]:
few_shot_prompt.invoke(input="What's 3 🍎 10?").to_messages()

[HumanMessage(content='3 🍎 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='15', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='7 🍎 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='21', additional_kwargs={}, response_metadata={})]

In [135]:
few_shot_prompt.invoke(input={"input":"horse"}).to_messages()

[HumanMessage(content='Write me a poem about the moon', additional_kwargs={}, response_metadata={}),
 AIMessage(content='One for the moon, and one for me, who are we to talk about the moon?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What did the cow say to the moon?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nothing at all', additional_kwargs={}, response_metadata={})]

In [136]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [138]:
chain.invoke({"input": "What's 20 🍎 30?"})

AIMessage(content='600  \n\nRemember, "🍎" means multiplication! 😊  \n', additional_kwargs={}, response_metadata={'model': 'gemma2:9b', 'created_at': '2024-10-09T13:27:47.397424Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2420239250, 'load_duration': 57965875, 'prompt_eval_count': 64, 'prompt_eval_duration': 646250000, 'eval_count': 17, 'eval_duration': 1709808000}, id='run-d2dc83d1-98ea-44c3-af60-efe1569c963f-0', usage_metadata={'input_tokens': 64, 'output_tokens': 17, 'total_tokens': 81})